<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Behavioral_Testing_with_CheckList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --force-reinstall ipykernel
!pip install checklist

In [ ]:
from checklist.test_suite import TestSuite
from checklist.editor import Editor
from checklist.perturb import Perturb
import numpy as np

# Assuming 'model' is your pre-trained model and 'tokenizer' is the corresponding tokenizer
# Define a prediction function
def predict_function(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()
    return predictions

# Initialize the test suite
suite = TestSuite()

# Example of adding tests using Checklist Editor and Perturb functions
editor = Editor()
suite.add(
    suite_name="Sentiment Analysis Tests",
    test=editor.template(
        "The movie was {adj}.",
        adj=["great", "terrible", "boring", "excellent", "awful"],
        labels=[1, 0, 0, 1, 0]  # Adjust labels as per your task (e.g., sentiment analysis)
    )
)

# Adding more perturbation-based tests (e.g., inserting typos, changing names)
perturbation_tests = Perturb.perturb(["The movie was great.", "The book was terrible."], Perturb.add_typos)
suite.add(
    suite_name="Perturbation Tests",
    test=perturbation_tests.data,
    labels=[1, 0] * (len(perturbation_tests.data) // 2)
)

# Run the tests and summarize results
suite.run(predict_function)
suite.summary()